In [2]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import PrintfTickFormatter
import statistics
import random
import numpy as np
import scipy as sp
import bokeh
import math
import re
from copy import deepcopy, copy

NUM_HOSTS = 2
HOSTS = ('Mac', 'other machine')
date_pattern = '%Y-%m-%dT%H-%M-%S'
date_pattern_new = '%Y-%m-%dT%H-%M-%S-%f'
things_to_extract_webrtc = {'chrome':
                        {'index': 0,
                         'name': 'chrome',
                         'line_dash': 'dashed',
                         'marker': 'circle',
                         'color': 'green'
                        }, 
                     'firefox':
                         {'index': 1,
                          'name': 'firefox',
                          'line_dash': 'solid',
                          'marker' : 'triangle', 
                          'color': 'red'
                         },
                     'ping':
                         {'index': 2,
                          'name': 'system ping',
                          'line_dash': 'dotted',
                          'marker': 'square',
                          'color': 'blue'
                        }
                    }
things_to_extract_time = {'chrome':
                        {'index': 0,
                         'name': 'chrome',
                         'line_dash': 'dashed',
                         'marker': 'circle',
                         'color': 'green'
                        }, 
                     'firefox':
                         {'index': 1,
                          'name': 'firefox',
                          'line_dash': 'solid',
                          'marker' : 'triangle', 
                          'color': 'red'
                         },
                     'safari':
                         {'index': 2,
                          'name': 'safari',
                          'line_dash': 'dotted',
                          'marker': 'square',
                          'color': 'blue'
                        }
                    }
pings_per_run = 2
color_per_host = ['blue', 'red']

measurement_types = ['ping', 'result']
markers_by_type = {'ping': 'circle', 'result': 'square'}
line_style_by_type = {'ping': 'solid', 'result': 'dashed'}
alpha_color = 0.3
line_width = 0.1
# all_css_colors = ["AliceBlue","AntiqueWhite","Aqua","Aquamarine","Azure","Beige","Bisque","Black","BlanchedAlmond","Blue","BlueViolet","Brown","BurlyWood","CadetBlue","Chartreuse","Chocolate","Coral","CornflowerBlue","Cornsilk","Crimson","Cyan","DarkBlue","DarkCyan","DarkGoldenRod","DarkGray","DarkGreen","DarkKhaki","DarkMagenta","DarkOliveGreen","Darkorange","DarkOrchid","DarkRed","DarkSalmon","DarkSeaGreen","DarkSlateBlue","DarkSlateGray","DarkTurquoise","DarkViolet","DeepPink","DeepSkyBlue","DimGray","DodgerBlue","FireBrick","FloralWhite","ForestGreen","Fuchsia","Gainsboro","GhostWhite","Gold","GoldenRod","Gray","Green","GreenYellow","HoneyDew","HotPink","IndianRed","Indigo","Ivory","Khaki","Lavender","LavenderBlush","LawnGreen","LemonChiffon","LightBlue","LightCoral","LightCyan","LightGoldenRodYellow","LightGray","LightGreen","LightPink","LightSalmon","LightSeaGreen","LightSkyBlue","LightSlateGray","LightSteelBlue","LightYellow","Lime","LimeGreen","Linen","Magenta","Maroon","MediumAquaMarine","MediumBlue","MediumOrchid","MediumPurple","MediumSeaGreen","MediumSlateBlue","MediumSpringGreen","MediumTurquoise","MediumVioletRed","MidnightBlue","MintCream","MistyRose","Moccasin","NavajoWhite","Navy","OldLace","Olive","OliveDrab","Orange","OrangeRed","Orchid","PaleGoldenRod","PaleGreen","PaleTurquoise","PaleVioletRed","PapayaWhip","PeachPuff","Peru","Pink","Plum","PowderBlue","Purple","Red","RosyBrown","RoyalBlue","SaddleBrown","Salmon","SandyBrown","SeaGreen","SeaShell","Sienna","Silver","SkyBlue","SlateBlue","SlateGray","Snow","SpringGreen","SteelBlue","Tan","Teal","Thistle","Tomato","Turquoise","Violet","Wheat","White","WhiteSmoke","Yellow","YellowGreen"]
# shuffled_colors = list(all_css_colors)
# random.shuffle(shuffled_colors)
# colors = ['aqua', 'black', 'blue', 'fuchsia', 'gray', 'green', 'lime', 'maroon', 'navy', 'olive', 'purple', 'red', 'silver', 'teal', 'yellow', 'orange', 'brown', 'violet', 'yellowgreen', 'darkorange', 'darksalmon', 'firebrick', 'indianred', 'khaki', 'magenta', 'cyan', 'springgreen', 'tomato', 'seagreen', 'limegreen', 'gold', 'chartreuse', 'aquamarine']
# shuffled_colors = list(colors)
# random.shuffle(shuffled_colors)
shuffled_colors = ['black','blue','green','purple','red','orange','silver','yellow','brown','magenta','cyan','olive']*10
line_styles = [((item+1)*2,(item+1)*2) for item in range(100)]

# Load all datasets

In [3]:
# %debug
import os
import io
import json
import datetime
import statistics
import sys
from functools import reduce

## WebRTC overnight at home one machine
# good_all_measurements_afternoon = (datetime.datetime.strptime('2016-04-26T13-00-00', date_pattern), 
#                                         datetime.datetime.strptime('2016-04-26T16-15-00', date_pattern))
# good_all_measurements_during_day = (datetime.datetime.strptime('2016-04-28T10-00-00', date_pattern), 
#                                         datetime.datetime.strptime('2016-04-28T11-45-00', date_pattern))
# good_all_measurements_overnight_chrome = (datetime.datetime.strptime('2016-04-28T19-15-00', date_pattern), 
#                                         datetime.datetime.strptime('2016-04-28T20-45-00', date_pattern))
# good_all_measurements_overnight_chrome_load = (datetime.datetime.strptime('2016-04-28T21-00-00', date_pattern), 
#                                         datetime.datetime.strptime('2016-04-28T23-00-00', date_pattern))
# good_all_measurements_overnight_firefox = (datetime.datetime.strptime('2016-04-28T23-00-00', date_pattern), 
#                                         datetime.datetime.strptime('2016-04-29T01-00-00', date_pattern))
# good_all_measurements_overnight_firefox_load = (datetime.datetime.strptime('2016-04-29T01-00-00', date_pattern), 
#                                         datetime.datetime.strptime('2016-04-29T03-00-00', date_pattern))


# job 32 at Fri Apr 29 18:57:20 2016
# job 33 at Fri Apr 29 20:57:00 2016
# job 34 at Fri Apr 29 22:57:00 2016
# job 35 at Sat Apr 30 00:57:00 2016
good_linux_overnight_chrome = (datetime.datetime.strptime('2016-04-29T19-15-00', date_pattern), 
                                        datetime.datetime.strptime('2016-04-29T20-45-00', date_pattern))
good_linux_overnight_chrome_load = (datetime.datetime.strptime('2016-04-29T21-15-00', date_pattern), 
                                        datetime.datetime.strptime('2016-04-29T22-45-00', date_pattern))
good_linux_overnight_firefox = (datetime.datetime.strptime('2016-04-29T23-15-00', date_pattern), 
                                        datetime.datetime.strptime('2016-04-30T00-45-00', date_pattern))
good_linux_overnight_firefox_load = (datetime.datetime.strptime('2016-04-30T01-15-00', date_pattern), 
                                        datetime.datetime.strptime('2016-04-30T02-45-00', date_pattern))

# job 48 at Sat Apr 30 16:03:59 2016
# job 49 at Sat Apr 30 18:03:00 2016
# job 50 at Sat Apr 30 20:03:00 2016
# job 51 at Sat Apr 30 22:03:00 2016
good_mac_overnight_firefox = (datetime.datetime.strptime('2016-04-30T20-15-00', date_pattern), 
                                        datetime.datetime.strptime('2016-04-30T21-45-00', date_pattern))
good_mac_overnight_firefox_load = (datetime.datetime.strptime('2016-04-30T22-15-00', date_pattern), 
                                        datetime.datetime.strptime('2016-04-30T23-45-00', date_pattern))

# 56 Tue May  3 13:40:00 2016
# 57 Tue May  3 15:40:00 2016
good_mac_again_chrome = (datetime.datetime.strptime('2016-05-03T14-00-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-03T15-45-00', date_pattern))
good_mac_again_chrome_load = (datetime.datetime.strptime('2016-05-03T16-00-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-03T17-45-00', date_pattern))

# job 66 at Wed May  4 19:08:25 2016
# job 67 at Wed May  4 21:08:00 2016
# job 76 at Thu May  5 15:55:44 2016
# job 77 at Thu May  5 17:55:00 2016

good_win_overnight_chrome = (datetime.datetime.strptime('2016-05-04T19-15-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-04T20-45-00', date_pattern))
good_win_overnight_chrome_load = (datetime.datetime.strptime('2016-05-04T21-15-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-04T22-45-00', date_pattern))

# Win Firefox
# job 82 at Thu May  5 19:45:12 2016
# job 83 at Thu May  5 21:45:00 2016
good_win_overnight_firefox = (datetime.datetime.strptime('2016-05-05T20-00-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-05T21-30-00', date_pattern))
good_win_overnight_firefox_load = (datetime.datetime.strptime('2016-05-05T22-00-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-05T23-30-00', date_pattern))

### Win and then Linux
# job 93 at Fri May  6 19:10:28 2016
# job 94 at Fri May  6 20:40:00 2016
# job 95 at Fri May  6 22:10:00 2016
# job 96 at Fri May  6 23:40:00 2016
# job 97 at Sat May  7 01:10:00 2016
# job 98 at Sat May  7 02:40:00 2016
# job 99 at Sat May  7 04:10:00 2016
# job 100 at Sat May  7 05:40:00 2016
short_win_overnight_chrome = (datetime.datetime.strptime('2016-05-06T19-15-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-06T20-30-00', date_pattern))
short_win_overnight_chrome_load = (datetime.datetime.strptime('2016-05-06T20-45-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-06T22-00-00', date_pattern))
short_win_overnight_firefox = (datetime.datetime.strptime('2016-05-06T22-15-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-06T23-30-00', date_pattern))
short_win_overnight_firefox_load = (datetime.datetime.strptime('2016-05-06T23-45-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-07T01-00-00', date_pattern))

### Linux and Mac
# job 101 at Sat May  7 20:25:50 2016
# job 102 at Sat May  7 21:55:00 2016
# job 103 at Sat May  7 23:25:00 2016
# job 104 at Sun May  8 00:55:00 2016
# job 105 at Sun May  8 02:25:00 2016
# job 106 at Sun May  8 03:55:00 2016
# job 107 at Sun May  8 05:25:00 2016
# job 108 at Sun May  8 06:55:00 2016

short_mac_overnight_chrome = (datetime.datetime.strptime('2016-05-07T20-30-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-07T21-45-00', date_pattern))
short_mac_overnight_chrome_load = (datetime.datetime.strptime('2016-05-07T22-00-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-07T23-15-00', date_pattern))
short_mac_overnight_firefox = (datetime.datetime.strptime('2016-05-07T23-30-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-08T00-45-00', date_pattern))
short_mac_overnight_firefox_load = (datetime.datetime.strptime('2016-05-08T01-00-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-08T02-15-00', date_pattern))

short_linux_overnight_chrome = (datetime.datetime.strptime('2016-05-08T02-30-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-08T03-45-00', date_pattern))
short_linux_overnight_chrome_load = (datetime.datetime.strptime('2016-05-08T04-00-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-08T05-15-00', date_pattern))
short_linux_overnight_firefox = (datetime.datetime.strptime('2016-05-08T05-30-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-08T06-45-00', date_pattern))
short_linux_overnight_firefox_load = (datetime.datetime.strptime('2016-05-08T07-00-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-09T08-15-00', date_pattern))

# New mac old
# job 134 at Wed May 11 20:06:59 2016
# job 135 at Wed May 11 20:36:00 2016
# job 136 at Wed May 11 21:06:00 2016
# job 137 at Wed May 11 21:36:00 2016
new2_mac_chrome = (datetime.datetime.strptime('2016-05-11T20-10-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-11T20-35-00', date_pattern))
new2_mac_chrome_load = (datetime.datetime.strptime('2016-05-11T20-40-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-11T21-05-00', date_pattern))
new2_mac_firefox = (datetime.datetime.strptime('2016-05-11T21-10-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-11T21-35-00', date_pattern))
new2_mac_firefox_load = (datetime.datetime.strptime('2016-05-11T21-40-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-11T22-05-00', date_pattern))

# job 178 at Wed May 11 23:27:33 2016
# job 179 at Wed May 11 23:57:00 2016
# job 180 at Thu May 12 00:27:00 2016
# job 181 at Thu May 12 00:57:00 2016
# job 182 at Thu May 12 01:27:00 2016
# job 183 at Thu May 12 01:57:00 2016
# job 184 at Thu May 12 02:27:00 2016
# job 185 at Thu May 12 02:57:00 2016
new_win_chrome = (datetime.datetime.strptime('2016-05-11T23-30-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-11T23-57-00', date_pattern))
new_win_chrome_load = (datetime.datetime.strptime('2016-05-12T00-00-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-12T00-27-00', date_pattern))
new_win_firefox = (datetime.datetime.strptime('2016-05-12T00-30-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-12T00-57-00', date_pattern))
new_win_firefox_load = (datetime.datetime.strptime('2016-05-12T01-00-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-12T01-27-00', date_pattern))

new_linux_chrome = (datetime.datetime.strptime('2016-05-12T01-32-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-12T01-57-00', date_pattern))
new_linux_chrome_load = (datetime.datetime.strptime('2016-05-12T02-02-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-12T02-27-00', date_pattern))
new_linux_firefox = (datetime.datetime.strptime('2016-05-12T02-32-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-12T02-57-00', date_pattern))
new_linux_firefox_load = (datetime.datetime.strptime('2016-05-12T03-02-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-12T03-27-00', date_pattern))

# New mac again
# job 186 at Thu May 12 10:35:53 2016
# job 187 at Thu May 12 11:05:00 2016
# job 188 at Thu May 12 11:35:00 2016
# job 189 at Thu May 12 12:05:00 2016

new_mac_chrome = (datetime.datetime.strptime('2016-05-12T10-40-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-12T11-05-00', date_pattern))
new_mac_chrome_load = (datetime.datetime.strptime('2016-05-12T11-10-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-12T11-35-00', date_pattern))
new_mac_firefox = (datetime.datetime.strptime('2016-05-12T11-40-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-12T12-05-00', date_pattern))
new_mac_firefox_load = (datetime.datetime.strptime('2016-05-12T12-10-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-12T12-40-00', date_pattern))

# Mac with Date.now()
# job 190 at Thu May 12 13:26:02 2016
new_mac_chrome_inaccurate = (datetime.datetime.strptime('2016-05-12T13-30-00', date_pattern), 
                                        datetime.datetime.strptime('2016-05-12T14-00-00', date_pattern))

full_set = set(map(lambda item: str(item), range(pings_per_run)))
property_to_color = {}
measured_things = None
data_point_count = 50

def extract_data(start_date=None, end_date=None, mode='all', num_hosts=NUM_HOSTS):
    global measured_things
    if isinstance(start_date, list) or isinstance(start_date, set) or isinstance(start_date, tuple):
        start_date, end_date = start_date
    if mode == 'fathom':
        d = 'collected_data'
    elif mode == 'webrtc_firefox':
        d = 'collected_data_webrtc_firefox'
    elif mode == 'webrtc_chrome':
        d = 'collected_data_webrtc_chrome'
    elif mode == 'webrtc_unified':
        d = 'collected_data_webrtc_unified'
    elif mode == 'time':
        d = 'collected_data_time'
#         d = 'collected_data_time_linux'
    elif mode == 'all':
        d = 'collected_data_webrtc_unified'
    else:
        d = 'collected_data_webrtc'
    extracted_data = []
    file_names = []
    files = [f for f in os.listdir(d) if os.path.isfile(os.path.join(d,f))]
#     start_timestamp = None
    if mode=='time':
        timestamps_start_times = [[None,None],[None,None],[None,None]]
        highres_timestamps_start_times = [[None,None],[None,None],[None,None]]
    for filename in sorted(files):
        if not filename.startswith('file'):
#             print("Skipped "+filename+" because it's some OS file")
            continue

        date_string = filename.split('_')[1].split('.')[0]
        file_date = datetime.datetime.strptime(date_string, date_pattern_new)
    
        if start_date and file_date < start_date:
            continue
        if end_date and file_date > end_date:
            break
    
        current_file = io.open(os.path.join(d,filename))
        content = current_file.read()

#         print("Processing "+filename)
        
        as_list = json.loads(content)
        if mode=='all' and not measured_things:
            measured_things = [','.join(item) for item in as_list['measurementOrder']]
        as_list['datetime'] = file_date

        current_file.close()
        file_names.append(filename)
        extracted_data.append(as_list)
#     print(dictionaries)
#     print(extracted_data)

    return extracted_data

# Plot

In [4]:
# %debug
from bokeh.io import output_notebook, output_file
from bokeh.plotting import figure, show, save, ColumnDataSource
from bokeh.models import PrintfTickFormatter, NumeralTickFormatter, HoverTool, Range1d
output_notebook()
# output_file('plots.html')

def get_distribution(things):
    distribution = {}
    for thing in things:
        if not thing in distribution:
            distribution[thing] = 0
        distribution[thing] += 1
    return distribution

def get_cumulative_distribution(dist):
    cumulative_distribution = {}
    value_from_previous = 0
    for thing in sorted(dist.keys()):
        cumulative_distribution[thing] = value_from_previous + dist[thing]
        value_from_previous = cumulative_distribution[thing]
    return cumulative_distribution

def flatten(l):
    return [item for sublist in l for item in sublist]

def merge_datasets(arrays, names):
    arrays = zip(*arrays)
    new_array = []
    for datasets in arrays:
        new_dataset = {}
        for i, item in enumerate(datasets):
            item = deepcopy(item)
            keys_to_iterate = deepcopy(list(item.keys()))
            for key in keys_to_iterate:
                if key != 'datetime' and key != 'ts':
                    new_dataset[key+'_merged:'+names[i]] = deepcopy(item[key])
                else:
                    new_dataset[key] = deepcopy(item[key])
            new_dataset.update(item)
        new_array.append(new_dataset)
    return new_array

def make_legend_text(thing, measurement_type=None,show_legend=True):
    if not show_legend:
        return ''
    result = thing.split('_')[0]
    if 'websocket' in thing:
        result = 'WebSocket'
    elif 'webrtc' in thing:
        result = 'WebRTC'
    elif 'ajax' in thing:
        result = 'AJAX'
        result = result+', '+thing.split(',')[1]
        result+=', '
        if '1337' in thing:
            result+='closed port'
        elif 'nvalid' in thing:
            result+='port 80, inexistent URL'
        elif 'disallowed' in thing: 
            result+='port 80, disallowed'
        elif 'allowed' in thing: 
            result+='port 80, allowed'
        else:
            result+='port 80'
    if measurement_type=='ping':
        result = 'Native ping'
    else: 
        merged_labels = thing.split('_merged:')[1:]
        for label in merged_labels:
            result += ', '+label
    return result

factor = 2
plot_width = 1000*factor
plot_height = 550*factor
# NUM_HOSTS = 1
ITEM_TO_TAKE = 1
### Style
LINE_WIDTH = 4
LABEL_FONT_SIZE = str(factor*25)+'pt'
TICK_FONT_SIZE = str(factor*20)+'pt'

def plot_data(dataset, desired_num_of_ticks=None, show_legend=True, legend_location=None, show_native_ping=True, plot_type='fathom', things=[], thing_regex='', num_hosts=NUM_HOSTS, plot_title="", machine_names=["Machine 1", "Machine 2"], what=['cdf', 'timeline'], x_range=None, y_range=None):

    if dataset.__class__.__name__ == 'tuple':
        extracted_data = extract_data(dataset, mode='all')
    else:
        extracted_data = dataset
#     import pdb; pdb.set_trace()
    
#     print('measured_things', measured_things)
    if plot_type=='all':
        things_to_extract = measured_things
    if things:
        things_to_extract = things
#     print('things_to_extract', things_to_extract)
    x = {}
    y = {}

    p = []
    p.append(figure(tools="pan,box_zoom,reset,save,wheel_zoom", x_axis_type="datetime", plot_width=plot_width, plot_height=plot_height))#, title=plot_title+" "+machine_names[0]))
    p.append(figure(tools="pan,box_zoom,reset,save,wheel_zoom", x_axis_type="datetime", plot_width=plot_width, plot_height=plot_height))#, title=plot_title+" "+machine_names[1]))
    if len(things_to_extract) > 10:
#         print('p', things_to_extract)
        p[0].title = plot_title+" "+machine_names[0]
        p[1].title = plot_title+" "+machine_names[1]
    p_cum = []
    p_cum.append(figure(tools="pan,box_zoom,reset,save,wheel_zoom", plot_width=plot_width, plot_height=plot_height))#, title=plot_title+" "+machine_names[0]))
    p_cum.append(figure(tools="pan,box_zoom,reset,save,wheel_zoom", plot_width=plot_width, plot_height=plot_height))#, title=plot_title+" "+machine_names[1]))
    if len(things_to_extract) > 10:
#         print('p_cum', things_to_extract)
        p_cum[0].title = plot_title+" "+machine_names[0]
        p_cum[1].title = plot_title+" "+machine_names[1]
#     p_diff = []
#     p_diff.append(figure(tools="pan,box_zoom,reset,save,wheel_zoom", x_axis_type="datetime", plot_width=plot_width, plot_height=plot_height))#, title=plot_title+" "+machine_names[0]))
#     p_diff.append(figure(tools="pan,box_zoom,reset,save,wheel_zoom", x_axis_type="datetime", plot_width=plot_width, plot_height=plot_height))#, title=plot_title+" "+machine_names[1]))

    for i in range(num_hosts):
        p[i].add_tools(HoverTool(tooltips=[("(x,y)", "($x, $y)"),("legend", "@desc")]))
        p[i].xaxis.axis_label="time"
        p[i].yaxis.axis_label="RTT"
        p[i].yaxis.formatter = PrintfTickFormatter(format="%.3f ms")
        p[i].xaxis.axis_label_text_font_size = LABEL_FONT_SIZE
        p[i].yaxis.axis_label_text_font_size = LABEL_FONT_SIZE
        p[i].xaxis.major_label_text_font_size = TICK_FONT_SIZE
        p[i].yaxis.major_label_text_font_size = TICK_FONT_SIZE
        if x_range:
            p[i].set(x_range=x_range)
        if y_range:
            p[i].set(y_range=y_range)
        
        p_cum[i].add_tools(HoverTool(tooltips=[("(x,y)", "($x, $y)"),("legend", "@desc")]))
        p_cum[i].yaxis.axis_label="CDF"
        p_cum[i].xaxis.axis_label_text_font_size = LABEL_FONT_SIZE
        p_cum[i].yaxis.axis_label_text_font_size = LABEL_FONT_SIZE
        p_cum[i].xaxis.major_label_text_font_size = TICK_FONT_SIZE
        p_cum[i].yaxis.major_label_text_font_size = TICK_FONT_SIZE
        if 'difference-cdf' in what:
            p_cum[i].xaxis.axis_label="Difference between measurement RTT and ICMP ping RTT"
        else:
            p_cum[i].xaxis.axis_label="RTT"
#         if not use_floating_point:
        p_cum[i].xaxis.formatter = PrintfTickFormatter(format="%d ms")
        if desired_num_of_ticks:
            p_cum[i].xaxis[0].ticker.desired_num_ticks = desired_num_of_ticks
#         else:
#             p_cum[i].xaxis.formatter = PrintfTickFormatter(format="%.1f ms")
#         p_cum[i].xaxis.formatter = NumeralTickFormatter(format="0.[0] ms")
        p_cum[i].yaxis.formatter = PrintfTickFormatter(format="%.1f")
        if x_range:
            p_cum[i].set(x_range=x_range)
        if y_range:
            p_cum[i].set(y_range=y_range)
        else:
            p_cum[i].set(y_range=Range1d(0.0,1.0))
        
#         p_diff[i].add_tools(HoverTool(tooltips=[("(x,y)", "($x, $y)"),("legend", "@desc")]))
#         p_diff[i].xaxis.axis_label="time"
#         p_diff[i].yaxis.axis_label="difference"
#         p_diff[i].yaxis[0].formatter = PrintfTickFormatter(format="%.3f")
#         if x_range:
#             p_diff[i].set(x_range=x_range)
#         if y_range:
#             p_diff[i].set(y_range=y_range)
        
#         things_for_maximum = []
        for thing in things_to_extract:
#             print("Yeah, something matched the regex!")
            for measurement_type in measurement_types:
#                 print(thing+','+measurement_type)
                if not thing+','+measurement_type in x:
                    x[thing+','+measurement_type] = [None,None]
                if not thing+','+measurement_type in y:
                    y[thing+','+measurement_type] = [None,None]
                if not thing+','+measurement_type+'_cumulative' in x:
                    x[thing+','+measurement_type+'_cumulative'] = [None,None]
                if not thing+','+measurement_type+'_cumulative' in y:
                    y[thing+','+measurement_type+'_cumulative'] = [None,None]
                if not thing+','+measurement_type+'_lost' in x:
                    x[thing+','+measurement_type+'_lost'] = [None,None]
                    
                if i==0:
                    continue
                    
#                 filtered_items_debug = [(item[thing][i]['ping']['data'], item[thing][i]['result']['data']) for item in extracted_data if thing in item and 
#                                   'ping' in item[thing][i] and 'result' in item[thing][i] and 
#                                   'data' in item[thing][i]['ping'] and 'data' in item[thing][i]['result'] and
#                                   len(item[thing][i]['ping']['data'])==2 and len(item[thing][i]['result']['data'])==2]
#                 import pdb; pdb.set_trace()
                filtered_items = [item for item in extracted_data if thing in item and 
                                  'ping' in item[thing][i] and 'result' in item[thing][i] and 
                                  'data' in item[thing][i]['ping'] and 'data' in item[thing][i]['result'] and
                                  not None in item[thing][i]['ping']['data'] and not None in item[thing][i]['result']['data'] and
                                  len(item[thing][i]['ping']['data']) > ITEM_TO_TAKE and len(item[thing][i]['result']['data']) > ITEM_TO_TAKE and
                                  'difference' in item[thing][i]['ping']['data'][ITEM_TO_TAKE] and
                                  'difference' in item[thing][i]['result']['data'][ITEM_TO_TAKE] and
                                  item[thing][i]['ping']['data'][ITEM_TO_TAKE]['difference'] and
                                  item[thing][i]['result']['data'][ITEM_TO_TAKE]['difference']]
                filtered_items = filtered_items[-data_point_count:]
#                 import pdb; pdb.set_trace()
                if len(filtered_items)<data_point_count:
                    print('Not enough data',len(filtered_items),'thing',thing)
#                 extracted_data = shortened_extracted_data
#                 if len(extracted_data)!=len(filtered_items):
#                     print('There was packet loss? :o')

#                 lost_packets = [item for item in extracted_data if not thing in item or 
#                                   not measurement_type in item[thing][i] or 
#                                   not 'data' in item[thing][i][measurement_type] or
#                                   None in item[thing][i][measurement_type]['data'] or 
#                                   not 'difference' in item[thing][i][measurement_type]['data'][ITEM_TO_TAKE] or 
#                                   not item[thing][i][measurement_type]['data'][ITEM_TO_TAKE]]
#                 x[thing+','+measurement_type+'_lost'].append(lost_packets)
#                 print('lost_packets', lost_packets)
#                 import pdb; pdb.set_trace()
                y_values = flatten([[m['difference'] for m in item[thing][i][measurement_type]['data'] if m!=None and m['difference']!=None and m['seqNum']!=None and m['seqNum']==ITEM_TO_TAKE] for item in filtered_items])
#                 y_values = flatten([[m['difference'] for m in item[thing][i][measurement_type]['data'] if m!=None and m['difference']!=None and m['seqNum']!=None] for item in filtered_items])
#                 print('thing',thing)
#                 if re.compile('observer').search(thing):
#                     y_values = [item-50 for item in y_values]
#                     print('Yeah, reducing')
                y[thing+','+measurement_type][i] = y_values
                if 'timeline' in what:
                    x_values = flatten([[item['datetime'] + datetime.timedelta(0,0.2*int(m['seqNum'])) + datetime.timedelta(0,1.0*[','.join(item) for item in item['measurementOrder']].index(thing)) for m in item[thing][i][measurement_type]['data'] if m!= None and m['difference']!=None and m['seqNum']!=None and m['seqNum']==ITEM_TO_TAKE] for item in filtered_items])
#                     x_values = flatten([[item['datetime'] + datetime.timedelta(0,0.2*int(m['seqNum'])) + datetime.timedelta(0,1.0*[','.join(item) for item in item['measurementOrder']].index(thing)) for m in item[thing][i][measurement_type]['data'] if m!= None and m['difference']!=None and m['seqNum']!=None] for item in filtered_items])
                    x[thing+','+measurement_type][i] = x_values
    
                if 'cdf' in what:
                    cum_dist = get_cumulative_distribution(get_distribution(y_values))
                    cum_dist = sorted(cum_dist.items())
                    y_cum = [item[1] for item in cum_dist]
                    m = max(y_cum)
                    y_cum = [(item/m) for item in y_cum]
                    y[thing+','+measurement_type+'_cumulative'][i] = y_cum
                    x[thing+','+measurement_type+'_cumulative'][i] = [item[0] for item in cum_dist]
#                     y[thing+','+measurement_type+'_cumulative'][i] = [0]+y_cum
#                     x[thing+','+measurement_type+'_cumulative'][i] = [0]+[item[0] for item in cum_dist]
                if measurement_type=='result' and 'difference-cdf' in what:
                    if len(y_values) != len(y[thing+',ping'][i]):
                        print('Result and ping are not the same length!')
                    cum_dist = get_cumulative_distribution(get_distribution([a-b for a,b in zip(y_values,y[thing+',ping'][i])]))
                    cum_dist = sorted(cum_dist.items())
                    y_cum = [item[1] for item in cum_dist]
                    m = max(y_cum)
                    y_cum = [(item/m) for item in y_cum]
                    y[thing+','+measurement_type+'_cumulative'][i] = y_cum
                    x[thing+','+measurement_type+'_cumulative'][i] = [item[0] for item in cum_dist]
#                     y[thing+','+measurement_type+'_cumulative'][i] = [0]+y_cum
#                     x[thing+','+measurement_type+'_cumulative'][i] = [0]+[item[0] for item in cum_dist]
                
#                 import pdb; pdb.set_trace()
            
#             if plot_type=='time':
#                 y[thing+'_measuredTime'].append([item[i][thing+'_measuredTime'] for item in extracted_data if item[i][thing+'_measuredTime']!=None])
#                 x[thing+'_measuredTime'].append([item[i]['datetime'] for item in extracted_data if item[i][thing+'_measuredTime']!=None])
#                 y[thing+'_inaccurateTime'].append([item[i][thing+'_inaccurateTime'] for item in extracted_data if item[i][thing+'_inaccurateTime']!=None])
#                 x[thing+'_inaccurateTime'].append([item[i]['datetime'] for item in extracted_data if item[i][thing+'_inaccurateTime']!=None])
                        
#                 things_for_maximum.extend(flatten(y[thing+','+measurement_type]))
#         maximum.append(max(things_for_maximum))

#     import pdb; pdb.set_trace()

    for i in range(num_hosts):
        if i==0:
            continue
        already_have_ping = False or not show_native_ping
        for thing in things_to_extract:
            for measurement_type in measurement_types:
                if thing_regex and not thing_regex.search(thing):
    #                     print('skipping', measurement_type, thing)
                    continue
    #                 p[i].scatter(x[thing+, thing_regex.search(thing)','+measurement_type][i], y[thing+','+measurement_type][i], legend=thing+','+measurement_type, fill_alpha=alpha_color, color=shuffled_colors[things_to_extract.index(thing)], marker=markers_by_type[measurement_type], line_color=None)
                if 'timeline' in what:
                    p[i].scatter('x', 'y', fill_alpha=alpha_color, color=shuffled_colors[things_to_extract.index(thing)], marker=markers_by_type[measurement_type], line_color=None, source=ColumnDataSource(data=dict(x=x[thing+','+measurement_type][i],y=y[thing+','+measurement_type][i],desc=[thing]*len(x[thing+','+measurement_type][i]))))
                if 'cdf' in what and (measurement_type=='result' or (measurement_type=='ping' and not already_have_ping)):
                    if measurement_type=='ping':
                        already_have_ping = True
#                     print('machine', i, 'thing', thing, 'measurement_type', measurement_type, 'x', x[thing+','+measurement_type+'_cumulative'][i], 'y', y[thing+','+measurement_type+'_cumulative'][i])
                    p_cum[i].line('x', 'y', line_width=factor*LINE_WIDTH, legend=make_legend_text(thing, measurement_type=measurement_type, show_legend=show_legend), line_color=shuffled_colors[things_to_extract.index(thing)+1 if measurement_type=='result' else 0], line_dash=factor*line_styles[things_to_extract.index(thing)+1 if measurement_type=='result' else 0], source=ColumnDataSource(data=dict(x=x[thing+','+measurement_type+'_cumulative'][i], y=y[thing+','+measurement_type+'_cumulative'][i],desc=[thing]*len(x[thing+','+measurement_type+'_cumulative'][i]))))
                if measurement_type=='ping':
                    continue
#                 p[i].multi_line(list(map(lambda x: [x,x], x[thing+','+measurement_type+'_lost'][i])), list(map(lambda x: [0.0, maximum[i]], x[thing+','+measurement_type+'_lost'][i])), line_dash=line_style_by_type[measurement_type], line_width=line_width, line_alpha=alpha_color/2, line_color=shuffled_colors[things_to_extract.index(thing)])
                if 'difference-cdf' in what:
                    p_cum[i].line('x', 'y', line_width=factor*LINE_WIDTH, legend=make_legend_text(thing, show_legend=show_legend), line_color=shuffled_colors[things_to_extract.index(thing)], line_dash=factor*line_styles[things_to_extract.index(thing)], source=ColumnDataSource(data=dict(x=x[thing+','+measurement_type+'_cumulative'][i], y=y[thing+','+measurement_type+'_cumulative'][i],desc=[thing]*len(x[thing+','+measurement_type+'_cumulative'][i]))))
                    
#                 [x/y for x, y in zip(bla, quak)]
#             p[i].line(x[thing+'_median'][i], y[thing+'_median'][i], line_color=things_to_extract[thing]['color'], line_width=line_width*8)
#                 print('maximum', maximum)
#             p[i].multi_line(list(map(lambda x: [x,x], x[thing+'_completely_lost'][i])), list(map(lambda x: [0.0, -maximum[i]/20], x[thing+'_completely_lost'][i])), line_width=line_width, line_alpha=1.0, line_color='purple')

    if 'timeline' in what:
#     show(p[0])
        p[1].legend.location = legend_location or "bottom_right"
#         p[1].legend.text_font_size='1pt'
        show(p[1])
    if 'difference-cdf' in what or 'cdf' in what:
#         import pdb; pdb.set_trace()
#         p_cum[0].legend.orientation = "bottom_right"
#         show(p_cum[0])
        p_cum[1].legend.location = legend_location or "bottom_right"
        p_cum[1].legend.label_text_font_size=str(factor*18)+'pt'
        p_cum[1].legend.legend_spacing=factor*12
        p_cum[1].legend.label_text_color='black'
        show(p_cum[1])
#     show(p_diff[0])
#     show(p_diff[1])

# dataset_to_use = good_all_measurements_overnight_chrome
# dataset_to_use = short_win_overnight_chrome
# dataset_to_use = short_mac_overnight_chrome
# dataset_to_use = good_mac_again_chrome
# dataset_to_use_load = short_mac_overnight_chrome_load

# ### Non-cooperative ones, with closed port, without disallowed
# things_internet_noncooperative_with_closed_port = ['ajax_ping,POST,observer:1337', 'ajax_ping,HEAD,observer:1337', 'ajax_ping,GET,observer:1337', 'ajax_ping,POST,observer/~dahora/someInvalidPath', 'ajax_ping,HEAD,observer/~dahora/someInvalidPath', 'ajax_ping,GET,observer/~dahora/someInvalidPath']
# plot_data(dataset_to_use, things=things_internet_noncooperative_with_closed_port, plot_type='all', plot_title='Non-cooperative measurements to Internet server from Chrome on', machine_names=['Mac 1', 'Linux'], what=['difference-cdf'], x_range=Range1d(2.5, 11))

# ### Non-cooperative ones, without closed port, without disallowed
# things_internet_noncooperative = ['ajax_ping,POST,observer/~dahora/someInvalidPath', 'ajax_ping,HEAD,observer/~dahora/someInvalidPath', 'ajax_ping,GET,observer/~dahora/someInvalidPath']
# plot_data(dataset_to_use, things=things_internet_noncooperative, plot_type='all', plot_title='Non-Cooperative measurements to Internet from Chrome on', machine_names=['Mac 1', 'Linux'], what=['difference-cdf'], x_range=Range1d(4, 9))

# ### Cooperative ones
# things_internet_cooperative = ['websocket_ping,observer:43234', 'webrtc_ping,observer', 'ajax_ping,HEAD,observer/~dahora/allowed', 'ajax_ping,POST,observer/~dahora/allowed', 'ajax_ping,GET,observer/~dahora/allowed']
# plot_data(dataset_to_use, things=things_internet_cooperative, plot_type='all', plot_title='Comparison coop. and non-coop. measurements to Internet from Chrome on', machine_names=['Mac', 'Linux'], what=['difference-cdf'], x_range=Range1d(0, 11))
# # plot_data(dataset_to_use, things=things_internet_cooperative, plot_type='all', plot_title='Comparison coop. and non-coop. measurements to Internet from Chrome on', machine_names=['Mac', 'Linux'], what=['timeline'])#, x_range=Range1d(1.5, 11))
# ### Best ones comparison non-cooperative, cooperative
# things_internet_comparison = ['websocket_ping,observer:43234', 'ajax_ping,POST,observer/~dahora/someInvalidPath', 'ajax_ping,POST,observer/~dahora/allowed']
# plot_data(dataset_to_use, things=things_internet_comparison, plot_type='all', plot_title='Comparison between best measurements to Internet from Chrome on', machine_names=['Mac 1', 'Linux'], what='cdf', x_range=Range1d(50, 66))

# ### Comparison router
# things_router = ['ajax_ping,POST,172.16.0.1:1337', 'ajax_ping,HEAD,172.16.0.1:1337', 'ajax_ping,HEAD,172.16.0.1/someInvalidPath/', 'ajax_ping,HEAD,172.16.0.1']
# plot_data(dataset_to_use, things=things_router, plot_type='all', plot_title='Measurements to home router from Chrome on', machine_names=['Mac 1', 'Linux'], what=['difference-cdf'], x_range=Range1d(1, 6.8))
# things_router_comparison = ['ajax_ping,POST,172.16.0.1:1337_merged:Chrome', 'ajax_ping,HEAD,172.16.0.1:1337_merged:Chrome', 'ajax_ping,POST,172.16.0.1:1337_merged:Firefox', 'ajax_ping,HEAD,172.16.0.1:1337_merged:Firefox']
# plot_data(merge_datasets([extract_data(short_mac_overnight_chrome), extract_data(short_mac_overnight_firefox)], ['Chrome', 'Firefox']), things=things_router_comparison, plot_type='all', plot_title='Measurements to home router from Chrome on', machine_names=['Mac', 'Linux'], what=['difference-cdf'], x_range=Range1d(1, 6))
# # plot_data(dataset_to_use, things=things_router, plot_type='all', plot_title='Measurements to home router from Chrome on', machine_names=['Mac 1', 'Linux'], what=['timeline'])

# ### Comparison other machine
# # things_other_machine = ['webrtc_ping,other_machine']
# # plot_data(dataset_to_use, things=things_other_machine, plot_type='all', plot_title='Measurements to other machine from Chrome on', machine_names=['Mac 1', 'Linux'], what='difference-cdf')#, x_range=Range1d(0, 7))
# things_other_machine_compare = ['webrtc_ping,other_machine_merged:Chrome', 'webrtc_ping,other_machine_merged:Firefox']
# plot_data(merge_datasets([extract_data(short_mac_overnight_chrome), extract_data(short_mac_overnight_firefox)], ['Chrome', 'Firefox']), things=things_other_machine_compare, plot_type='all', plot_title='Measurements to other machine from Chrome on', machine_names=['Mac 1', 'Linux'], what=['difference-cdf'], x_range=Range1d(0, 4))

# ### Weird Firefox, compare to Chrome
# things_weird_firefox = ['ajax_ping,POST,observer:1337_merged:Chrome', 'ajax_ping,POST,observer:1337_merged:Firefox']
# plot_data(merge_datasets([extract_data(short_mac_overnight_firefox), extract_data(short_mac_overnight_chrome)], ['Firefox', 'Chrome']), things=things_weird_firefox, plot_type='all', plot_title='Comparison between best measurements to Internet from Chrome on', machine_names=['Mac', 'Linux'], what=['cdf'])#, x_range=Range1d(50, 59))

# ### Load Chrome
# things_compare_load = ['ajax_ping,POST,172.16.0.1:1337_merged:Idle', 'ajax_ping,POST,172.16.0.1:1337_merged:Load', 'ajax_ping,POST,observer/~dahora/allowed_merged:Idle', 'ajax_ping,POST,observer/~dahora/allowed_merged:Load', 'ajax_ping,POST,observer:1337_merged:Idle', 'ajax_ping,POST,observer:1337_merged:Load', 'webrtc_ping,other_machine_merged:Idle', 'webrtc_ping,other_machine_merged:Load', 'webrtc_ping,other_machine_merged:Idle', 'webrtc_ping,other_machine_merged:Load', 'websocket_ping,observer:43234_merged:Idle', 'websocket_ping,observer:43234_merged:Load']
# plot_data(merge_datasets([extract_data(short_mac_overnight_chrome), extract_data(short_mac_overnight_chrome_load)], ['Idle', 'Load']), things=things_compare_load, plot_type='all', plot_title='Comparison idle vs load on Chrome on', machine_names=['Mac', 'Mac'], what=['cdf'], show_native_ping=False)
# # plot_data(merge_datasets([extract_data(good_mac_again_chrome), extract_data(good_mac_again_chrome_load)], ['Idle', 'Load']), things=things_compare_load, plot_type='all', plot_title='Comparison idle vs load on Chrome on', machine_names=['Mac', 'Mac'], what=['cdf'], show_native_ping=False)

# plot_data(merge_datasets([extract_data(short_linux_overnight_chrome), extract_data(short_linux_overnight_chrome_load)], ['Idle', 'Load']), things=things_compare_load, plot_type='all', plot_title='Comparison idle vs load on Chrome on', machine_names=['Mac', 'Linux'], what=['cdf'], show_native_ping=False)

# plot_data(merge_datasets([extract_data(short_win_overnight_chrome), extract_data(short_win_overnight_chrome_load)], ['Idle', 'Load']), things=[item for item in things_compare_load if not '1337' in item], plot_type='all', plot_title='Comparison idle vs load on Chrome on', machine_names=['Mac', 'Win'], what=['cdf'], show_native_ping=False)

# ### Load Firefox
# plot_data(merge_datasets([extract_data(short_mac_overnight_firefox), extract_data(short_mac_overnight_firefox_load)], ['Idle', 'Load']), things=things_compare_load, plot_type='all', plot_title='Comparison idle vs load on Firefox on', machine_names=['Mac', 'Mac'], what=['cdf'], show_native_ping=False)

# plot_data(merge_datasets([extract_data(short_linux_overnight_firefox), extract_data(short_linux_overnight_firefox_load)], ['Idle', 'Load']), things=things_compare_load, plot_type='all', plot_title='Comparison idle vs load on Firefox on', machine_names=['Mac', 'Linux'], what=['cdf'], show_native_ping=False)

# plot_data(merge_datasets([extract_data(short_win_overnight_firefox), extract_data(short_win_overnight_firefox_load)], ['Idle', 'Load']), things=[item for item in things_compare_load if not '1337' in item], plot_type='all', plot_title='Comparison idle vs load on Firefox on', machine_names=['Mac', 'Win'], what=['cdf'], show_native_ping=False)

# things_compare_load_browsers = ['websocket_ping,observer:43234_merged:Idle_merged:Firefox', 'websocket_ping,observer:43234_merged:Load_merged:Firefox', 'websocket_ping,observer:43234_merged:Idle_merged:Chrome', 'websocket_ping,observer:43234_merged:Load_merged:Chrome']
# things_compare_load_browsers = ['ajax_ping,POST,observer/~dahora/allowed_merged:Idle_merged:Chrome', 'ajax_ping,POST,observer/~dahora/allowed_merged:Load_merged:Chrome', 'ajax_ping,POST,observer/~dahora/allowed_merged:Idle_merged:Firefox', 'ajax_ping,POST,observer/~dahora/allowed_merged:Load_merged:Firefox', 'websocket_ping,observer:43234_merged:Idle_merged:Firefox', 'websocket_ping,observer:43234_merged:Load_merged:Firefox', 'websocket_ping,observer:43234_merged:Idle_merged:Chrome', 'websocket_ping,observer:43234_merged:Load_merged:Chrome']
# things_compare_load_browsers = ['ajax_ping,POST,observer/~dahora/allowed_merged:Idle', 'ajax_ping,POST,observer/~dahora/allowed_merged:Load', 'websocket_ping,observer:43234_merged:Idle', 'websocket_ping,observer:43234_merged:Load']
# plot_data(merge_datasets([extract_data(short_linux_overnight_firefox), extract_data(short_linux_overnight_firefox_load)], ['Idle', 'Load']), things=things_compare_load_browsers, plot_type='all', plot_title='Comparison idle vs load on Firefox on', machine_names=['Mac', 'Linux'], what=['cdf'], show_native_ping=False, x_range=Range1d(51, 60))

# no_ws = re.compile('^((?!WS).)*$')
# plot_data(short_linux_overnight_chrome, plot_type='all', plot_title='Chrome - Chrome', machine_names=['Mac', 'Linux'], thing_regex=no_ws)
# plot_data(short_linux_overnight_chrome_load, plot_type='all', plot_title='Chrome - Chrome load', machine_names=['Mac', 'Linux'], thing_regex=no_ws)
# plot_data(short_linux_overnight_firefox, plot_type='all', plot_title='Chrome - Firefox', machine_names=['Mac', 'Linux'], thing_regex=no_ws)
# plot_data(short_linux_overnight_firefox_load, plot_type='all', plot_title='Chrome - Firefox load', machine_names=['Mac', 'Linux'], thing_regex=no_ws)
# # plot_data(good_mac_again_chrome, plot_type='all', plot_title='Chrome - Chrome', machine_names=['Mac', 'Mac'], thing_regex=no_ws, what='cdf-time')
# # plot_data(good_mac_again_chrome_load, plot_type='all', plot_title='Chrome - Chrome load', machine_names=['Mac', 'Mac'], thing_regex=no_ws, what='cdf-time')
# # plot_data(good_mac_overnight_firefox, plot_type='all', plot_title='Chrome - Firefox', machine_names=['Mac', 'Mac'], thing_regex=no_ws, what='cdf-time')
# # plot_data(good_mac_overnight_firefox_load, plot_type='all', plot_title='Chrome - Firefox load', machine_names=['Mac', 'Mac'], thing_regex=no_ws, what='cdf-time')
# plot_data(short_win_overnight_chrome, plot_type='all', plot_title='Chrome - Chrome', machine_names=['Mac', 'Windows'], thing_regex=no_ws, what='cdf-time')
# plot_data(short_win_overnight_chrome_load, plot_type='all', plot_title='Chrome - Chrome load', machine_names=['Mac', 'Windows'], thing_regex=no_ws, what='cdf-time')
# plot_data(short_win_overnight_firefox, plot_type='all', plot_title='Chrome - Firefox', machine_names=['Mac', 'Windows'], thing_regex=no_ws, what='cdf-time')
# plot_data(short_win_overnight_firefox_load, plot_type='all', plot_title='Chrome - Firefox load', machine_names=['Mac', 'Windows'], thing_regex=no_ws, what='cdf-time')


# try:
#     all_merged
# except:
# all_datasets_chrome = [extract_data(item) for item in [short_mac_overnight_chrome, short_linux_overnight_chrome, short_win_overnight_chrome]]
# merged_chrome = merge_datasets(all_datasets_chrome, ['Mac', 'Linux', 'Win'])
# all_datasets_ff = [extract_data(item) for item in [short_mac_overnight_firefox, short_linux_overnight_firefox, short_win_overnight_firefox]]
# merged_ff = merge_datasets(all_datasets_ff, ['Mac', 'Linux', 'Win'])
# all_merged = merge_datasets([merged_chrome, merged_ff], ['Chrome', 'Firefox'])

# things_to_show_other_machine = ['webrtc_ping,other_machine_merged:Mac_merged:Chrome', 'webrtc_ping,other_machine_merged:Linux_merged:Chrome', 'webrtc_ping,other_machine_merged:Win_merged:Chrome', 'webrtc_ping,other_machine_merged:Mac_merged:Firefox', 'webrtc_ping,other_machine_merged:Linux_merged:Firefox', 'webrtc_ping,other_machine_merged:Win_merged:Firefox']
# plot_data(all_merged, things=things_to_show_other_machine, plot_type='all', plot_title='Comparison idle vs load on Firefox on', machine_names=['Mac', 'Linux'], what=['difference-cdf'], show_native_ping=False)#, x_range=Range1d(52, 65))

Loading BokehJS ...

In [10]:
# ### Non-cooperative ones, without closed port, without disallowed
# print('Uncoop old')
# things_internet_noncooperative = ['ajax_ping,POST,observer/~dahora/someInvalidPath', 'ajax_ping,HEAD,observer/~dahora/someInvalidPath', 'ajax_ping,GET,observer/~dahora/someInvalidPath']
# plot_data(new_mac_chrome, things=things_internet_noncooperative, plot_type='all', plot_title='Non-Cooperative measurements to Internet from Chrome on', machine_names=['Mac 1', 'Linux'], what=['difference-cdf'])#, x_range=Range1d(4, 9))

# ### Non-cooperative ones, without closed port, without disallowed
# print('Uncoop new')
# things_internet_noncooperative = ['ajax_ping,POST,observer/~dahora/someInvalidPath', 'ajax_ping,HEAD,observer/~dahora/someInvalidPath', 'ajax_ping,POST,observer/~dahora/disallowed', 'ajax_ping,HEAD,observer/~dahora/disallowed', 'ajax_ping,POST,observer:1337', 'ajax_ping,HEAD,observer:1337']
# plot_data(new_mac_chrome, things=things_internet_noncooperative, plot_type='all', plot_title='Non-Cooperative measurements to Internet from Chrome on', machine_names=['Mac 1', 'Linux'], what=['difference-cdf'])#, x_range=Range1d(4, 9))

# ### Weird Firefox, compare to Chrome
# print('Weird firefox')
# things_weird_firefox = ['ajax_ping,POST,observer:1337_merged:Chrome', 'ajax_ping,POST,observer:1337_merged:Firefox']
# plot_data(merge_datasets([extract_data(new_linux_firefox), extract_data(new_linux_chrome)], ['Firefox', 'Chrome']), things=things_weird_firefox, plot_type='all', plot_title='Comparison between best measurements to Internet from Chrome on', machine_names=['Mac', 'Linux'], what=['cdf'])#, x_range=Range1d(50, 220))

# ### Cooperative ones
# print('Coop')
# things_internet_cooperative = ['websocket_ping,observer:43234', 'webrtc_ping,observer', 'ajax_ping,HEAD,observer/~dahora/allowed', 'ajax_ping,POST,observer/~dahora/allowed', 'ajax_ping,GET,observer/~dahora/allowed']
# plot_data(new_mac_chrome, things=things_internet_cooperative, plot_type='all', plot_title='Comparison coop. and non-coop. measurements to Internet from Chrome on', machine_names=['Mac', 'Linux'], what=['difference-cdf'])#, x_range=Range1d(0, 11))
### Best ones comparison non-cooperative, cooperative
things_internet_comparison = ['websocket_ping,observer:43234', 'ajax_ping,POST,observer:1337', 'ajax_ping,POST,observer/~dahora/someInvalidPath', 'ajax_ping,POST,observer/~dahora/allowed', 'ajax_ping,POST,observer/~dahora/disallowed', 'webrtc_ping,observer']
plot_data(new_mac_chrome, things=things_internet_comparison, plot_type='all', plot_title='Comparison between best measurements to Internet from Chrome on', machine_names=['Mac 1', 'Linux'], what=['difference-cdf'])#, x_range=Range1d(50, 66))
# plot_data(new_mac_chrome, things=things_internet_comparison, plot_type='all', plot_title='Comparison between best measurements to Internet from Chrome on', machine_names=['Mac 1', 'Linux'], what=['timeline'])#, x_range=Range1d(50, 66))

# ### Comparison router
# # print('Router')
# # things_router = ['ajax_ping,POST,172.16.0.1:1337', 'ajax_ping,HEAD,172.16.0.1:1337', 'ajax_ping,HEAD,172.16.0.1/someInvalidPath/', 'ajax_ping,HEAD,172.16.0.1']
# # plot_data(new_mac_chrome, things=things_router, plot_type='all', plot_title='Measurements to home router from Chrome on', machine_names=['Mac 1', 'Linux'], what=['difference-cdf'])#, x_range=Range1d(1, 6.8))
# print('Compare chrome firefox regarding head and post')
# things_router_comparison = ['ajax_ping,POST,172.16.0.1:1337_merged:Chrome', 'ajax_ping,HEAD,172.16.0.1:1337_merged:Chrome', 'ajax_ping,GET,172.16.0.1:1337_merged:Chrome', 'ajax_ping,POST,172.16.0.1:1337_merged:Firefox', 'ajax_ping,HEAD,172.16.0.1:1337_merged:Firefox', 'ajax_ping,GET,172.16.0.1:1337_merged:Firefox']
# plot_data(merge_datasets([extract_data(new_linux_chrome), extract_data(new_linux_firefox)], ['Chrome', 'Firefox']), things=things_router_comparison, plot_type='all', plot_title='Measurements to home router from Chrome on', machine_names=['Mac', 'Linux'], what=['difference-cdf'])#, x_range=Range1d(1, 6))

# ### Weird Windows
# print('Weird Win')
# things_weird_windows = ['ajax_ping,POST,172.16.0.1:1337_merged:Windows', 'ajax_ping,POST,172.16.0.1:1337_merged:Mac']
# plot_data(merge_datasets([extract_data(new_mac_chrome), extract_data(new_win_chrome)], ['Mac', 'Windows']), things=things_weird_windows, plot_type='all', plot_title='Comparison between best measurements to Internet from Chrome on', machine_names=['Mac', 'Linux'], what=['cdf'], show_native_ping=False, legend_location='top_right')#, x_range=Range1d(0, 1050))

# ### Comparison other machine
# print('Other machine')
# # things_other_machine = ['webrtc_ping,other_machine']
# # plot_data(dataset_to_use, things=things_other_machine, plot_type='all', plot_title='Measurements to other machine from Chrome on', machine_names=['Mac 1', 'Linux'], what='difference-cdf')#, x_range=Range1d(0, 7))
# things_other_machine_compare = ['webrtc_ping,other_machine_merged:Chrome', 'webrtc_ping,other_machine_merged:Firefox']
# plot_data(merge_datasets([extract_data(new_mac_chrome), extract_data(new_mac_firefox)], ['Chrome', 'Firefox']), things=things_other_machine_compare, desired_num_of_ticks=5, plot_type='all', plot_title='Measurements to other machine from Chrome on', machine_names=['Mac 1', 'Linux'], what=['difference-cdf'])#, x_range=Range1d(0, 4))

# ### Load
# print('Firefox load')
# things_compare_load_browsers = ['ajax_ping,POST,observer/~dahora/allowed_merged:Idle', 'ajax_ping,POST,observer/~dahora/allowed_merged:Load', 'websocket_ping,observer:43234_merged:Idle', 'websocket_ping,observer:43234_merged:Load']
# plot_data(merge_datasets([extract_data(new_win_firefox), extract_data(new_win_firefox_load)], ['Idle', 'Load']), things=things_compare_load_browsers, plot_type='all', plot_title='Comparison idle vs load on Firefox on', machine_names=['Mac', 'Linux'], what=['difference-cdf'], show_native_ping=False)#, x_range=Range1d(51, 60))

In [6]:
def print_statistics(dataset, things_to_show=[]):
    extracted_data = extract_data(dataset, mode='all')
#     things_to_show = measured_things
#     print("Total datasets:", len(extracted_data))
    stats = {}
    for thing in things_to_show:
        for i in range(NUM_HOSTS):
            if i == 0:
                continue
            
            y = {}
            for measurement_type in measurement_types:
                filtered_items = [item for item in extracted_data if thing in item and 
                                  'ping' in item[thing][i] and 'result' in item[thing][i] and 
                                  'data' in item[thing][i]['ping'] and 'data' in item[thing][i]['result'] and
                                  len(item[thing][i]['ping']['data'])==2 and len(item[thing][i]['result']['data'])==2 and
                                  not None in item[thing][i]['ping']['data'] and not None in item[thing][i]['result']['data'] and
                                  'difference' in item[thing][i]['ping']['data'][0] and 'difference' in item[thing][i]['ping']['data'][1] and
                                  'difference' in item[thing][i]['result']['data'][0] and 'difference' in item[thing][i]['result']['data'][1] and
                                  item[thing][i]['ping']['data'][0]['difference'] and item[thing][i]['ping']['data'][1]['difference'] and
                                  item[thing][i]['result']['data'][0]['difference'] and item[thing][i]['result']['data'][1]['difference']]
#                 if len(filtered_items) != len(extracted_data):
#                     print('Had invalid data',len(filtered_items),'thing',thing,'len(extracted_data)', len(extracted_data))

                filtered_items = filtered_items[-data_point_count:]
                if len(filtered_items)<data_point_count:
                    print('Not enough data',len(filtered_items),'thing',thing,'len(extracted_data)', len(extracted_data))
#                     print([item[i][thing] for item in extracted_data])
                    
                y[measurement_type] = flatten([[m['difference'] for m in item[thing][i][measurement_type]['data'] if m!=None and m['difference']!=None and m['seqNum']!=None and m['seqNum']!=0] for item in filtered_items])
                if measurement_type=='ping':
                    continue
                else:
#                     y_values = y[measurement_type]
                    y_values = [a-b for a,b in zip(y[measurement_type],y['ping'])]

    #             import pdb; pdb.set_trace()
#                 stats[thing+'_mean'] = statistics.mean(y_values)
    #             print(thing+'_mean',stats[thing+'_mean'])
    #             stats[thing+'_stdev'] = statistics.stdev(y_values)
    #             print(thing+'_stdev',stats[thing+'_stdev'])
                stats[thing+'_median'] = [np.percentile(y_values,50),np.percentile(y_values,5),np.percentile(y_values,95)]
    #             print(thing+'_median',stats[thing+'_median'])
    #             stats[thing+'_min'] = min(y_values)
    #             print(thing+'_min',stats[thing+'_min'])
#     print("median")
    median_to_print = [str(round(m[0][0],1))+' ['+str(round(m[0][1],1))+', '+str(round(m[0][2],1))+'] '+m[1].split('_median')[0] for m in sorted([(b,a) for a,b in stats.items() if re.compile('median').search(a)])]
    print("\n".join(median_to_print[:3]))
#     print("\n".join(median_to_print))

#     print("mean")
#     print("\n".join([(" ".join([str(x) for x in m])) for m in sorted([(b,a) for a,b in stats.items() if re.compile('mean').search(a)])]))

all_things_router = ['ajax_ping,POST,172.16.0.1:1337', 'ajax_ping,HEAD,172.16.0.1:1337', 'ajax_ping,GET,172.16.0.1:1337', 'ajax_ping,POST,172.16.0.1/someInvalidPath/', 'ajax_ping,HEAD,172.16.0.1/someInvalidPath/', 'ajax_ping,GET,172.16.0.1/someInvalidPath/', 'ajax_ping,POST,172.16.0.1', 'ajax_ping,HEAD,172.16.0.1', 'ajax_ping,GET,172.16.0.1']
all_things_other_machine = ['webrtc_ping,other_machine']
all_things_server = ['ajax_ping,POST,observer:1337', 'ajax_ping,HEAD,observer:1337', 'ajax_ping,GET,observer:1337', 'ajax_ping,POST,observer/~dahora/someInvalidPath', 'ajax_ping,HEAD,observer/~dahora/someInvalidPath', 'ajax_ping,GET,observer/~dahora/someInvalidPath', 'websocket_ping,observer:43234', 'webrtc_ping,observer', 'ajax_ping,HEAD,observer/~dahora/allowed', 'ajax_ping,POST,observer/~dahora/allowed', 'ajax_ping,GET,observer/~dahora/allowed']

print('Mac')
print('chrome')
print('router')
print_statistics(new_mac_chrome, things_to_show=all_things_router)
# print_statistics(new_mac_chrome_load, things_to_show=all_things_router)
print('other')
print_statistics(new_mac_chrome, things_to_show=all_things_other_machine)
# print_statistics(new_mac_chrome_load, things_to_show=all_things_other_machine)
print('server')
print_statistics(new_mac_chrome, things_to_show=all_things_server)
# print_statistics(new_mac_chrome_load, things_to_show=all_things_server)
print('firefox')
print('router')
print_statistics(new_mac_firefox, things_to_show=all_things_router)
# print_statistics(new_mac_firefox_load, things_to_show=all_things_router)
print('other')
print_statistics(new_mac_firefox, things_to_show=all_things_other_machine)
# print_statistics(new_mac_firefox_load, things_to_show=all_things_other_machine)
print('server')
print_statistics(new_mac_firefox, things_to_show=all_things_server)
# print_statistics(new_mac_firefox_load, things_to_show=all_things_server)

# print("Date.now()")
# print('router')
# print_statistics(new_mac_chrome_inaccurate, things_to_show=all_things_router)
# # print_statistics(new_mac_chrome_load, things_to_show=all_things_router)
# print('other')
# print_statistics(new_mac_chrome_inaccurate, things_to_show=all_things_other_machine)
# # print_statistics(new_mac_chrome_load, things_to_show=all_things_other_machine)
# print('server')
# print_statistics(new_mac_chrome_inaccurate, things_to_show=all_things_server)
# # print_statistics(new_mac_chrome_load, things_to_show=all_things_server)

print('Linux')
print('chrome')
print('router')
print_statistics(new_linux_chrome, things_to_show=all_things_router)
# print_statistics(new_linux_chrome_load, things_to_show=all_things_router)
print('other')
print_statistics(new_linux_chrome, things_to_show=all_things_other_machine)
# print_statistics(new_linux_chrome_load, things_to_show=all_things_other_machine)
print('server')
print_statistics(new_linux_chrome, things_to_show=all_things_server)
# print_statistics(new_linux_chrome_load, things_to_show=all_things_server)
print('firefox')
print('router')
print_statistics(new_linux_firefox, things_to_show=all_things_router)
# print_statistics(new_linux_firefox_load, things_to_show=all_things_router)
print('other')
print_statistics(new_linux_firefox, things_to_show=all_things_other_machine)
# print_statistics(new_linux_firefox_load, things_to_show=all_things_other_machine)
print('server')
print_statistics(new_linux_firefox, things_to_show=all_things_server)
# print_statistics(new_linux_firefox_load, things_to_show=all_things_server)

print('Windows')
print('chrome')
print('router')
print_statistics(new_win_chrome, things_to_show=all_things_router)
# print_statistics(new_win_chrome_load, things_to_show=all_things_router)
print('other')
print_statistics(new_win_chrome, things_to_show=all_things_other_machine)
# print_statistics(new_win_chrome_load, things_to_show=all_things_other_machine)
print('server')
print_statistics(new_win_chrome, things_to_show=all_things_server)
# print_statistics(new_win_chrome_load, things_to_show=all_things_server)
print('firefox')
print('router')
print_statistics(new_win_firefox, things_to_show=all_things_router)
# print_statistics(new_win_firefox_load, things_to_show=all_things_router)
print('other')
print_statistics(new_win_firefox, things_to_show=all_things_other_machine)
# print_statistics(new_win_firefox_load, things_to_show=all_things_other_machine)
print('server')
print_statistics(new_win_firefox, things_to_show=all_things_server)
# print_statistics(new_win_firefox_load, things_to_show=all_things_server)

Mac
chrome
router
1.9 [1.8, 2.2] ajax_ping,POST,172.16.0.1:1337
2.1 [2.0, 4.5] ajax_ping,HEAD,172.16.0.1:1337
2.5 [2.1, 4.6] ajax_ping,GET,172.16.0.1:1337
other
2.3 [1.7, 3.6] webrtc_ping,other_machine
server
2.3 [2.0, 2.6] ajax_ping,POST,observer:1337
2.4 [2.0, 4.4] ajax_ping,HEAD,observer:1337
2.6 [2.4, 3.7] websocket_ping,observer:43234
firefox
router
3.1 [2.9, 3.4] ajax_ping,POST,172.16.0.1:1337
3.1 [2.9, 3.6] ajax_ping,GET,172.16.0.1:1337
3.2 [2.9, 3.8] ajax_ping,HEAD,172.16.0.1:1337
other
1.8 [1.5, 2.4] webrtc_ping,other_machine
server
2.4 [2.0, 3.5] websocket_ping,observer:43234
2.7 [2.4, 2.9] webrtc_ping,observer
3.8 [3.4, 55.1] ajax_ping,GET,observer:1337
Linux
chrome
router
4.0 [3.3, 4.7] ajax_ping,POST,172.16.0.1:1337
4.4 [3.9, 5.9] ajax_ping,HEAD,172.16.0.1:1337
4.9 [3.8, 5.8] ajax_ping,GET,172.16.0.1:1337
other
2.7 [2.2, 3.1] webrtc_ping,other_machine
server
2.8 [2.5, 4.0] websocket_ping,observer:43234
3.7 [3.4, 4.6] webrtc_ping,observer
4.1 [3.8, 4.6] ajax_ping,POST,obser

In [7]:
# %debug
def print_table(dataset, thing_regex='', os=None):
    if dataset.__class__.__name__ == 'list':
        extracted_data = []
        for d in dataset:
            extracted_data.extend(extract_data(d, mode='all')[:data_point_count])
    else:
        extracted_data = extract_data(dataset, mode='all')
        extracted_data = extracted_data[:data_point_count]
    things_to_show = measured_things
#     print("Total datasets:", len(extracted_data))
#     print("\\toprule\nOS & router & other device & server \\\\\n\\midrule")
#     print("\\toprule\nOS & router & other device & server \\\\")
    for i in range(NUM_HOSTS):
        if i==0:
            continue
        data = [[],[],[]]
        for thing in things_to_show:
            if thing_regex and not thing_regex.search(thing):
                continue

            measurement_type = 'ping'
#             import pdb; pdb.set_trace()
            y_values = flatten([[m['difference'] for m in item[thing][i][measurement_type]['data'] if m!=None and m['difference']!=None and m['seqNum']!=None and m['seqNum']!=0] for item in extracted_data])

            if '172' in thing:
                data[0].extend(y_values)
            elif 'other_machine' in thing:
                data[1].extend(y_values)
            elif 'observer' in thing:
                data[2].extend(y_values)
            else:
                raise Exception('Oops')

        print(' & '.join([os] + ['{[$'+str(round(min(l),1))+'$, $'+str(round(statistics.median(l),1))+'$, $'+str(round(max(l),1))+'$]}' for l in data]),end='')
#         print(' & '.join([os] + ['{[$'+str(round(min(l),1))+'$, $'+str(round(max(l),1))+'$]}' for l in data]),end='')
        print('\\\\')
#     print('\\bottomrule')
        
#         print('median', statistics.median(to_router))
    
#     print("median")
#     print("\n".join([(" ".join([str(x) for x in m])) for m in sorted([(b,a) for a,b in stats.items() if re.compile('median').search(a)])]))
#     print("mean")
#     print("\n".join([(" ".join([str(x) for x in m])) for m in sorted([(b,a) for a,b in stats.items() if re.compile('mean').search(a)])]))
    
print("\\toprule\nOS & Router & Other device & Server \\\\\n\\midrule")
print_table([new_mac_chrome,new_mac_chrome_load,new_mac_firefox,new_mac_firefox_load], os="OS X", thing_regex=re.compile('^((?!WS).)*$'))
print_table([new_linux_chrome,new_linux_chrome_load,new_linux_firefox,new_linux_firefox_load], os="Ubuntu", thing_regex=re.compile('^((?!WS).)*$'))
print_table([new_win_chrome,new_win_chrome_load,new_win_firefox,new_win_firefox_load], os="Win.", thing_regex=re.compile('^((?!WS).)*$'))
print('\\bottomrule')

\toprule
OS & Router & Other device & Server \\
\midrule
OS X & {[$0.2$, $0.4$, $0.9$]} & {[$0.3$, $0.6$, $1.0$]} & {[$50.4$, $50.8$, $52.6$]}\\
Ubuntu & {[$0.2$, $0.2$, $0.4$]} & {[$0.3$, $0.6$, $2.1$]} & {[$50.2$, $50.5$, $51.3$]}\\
Win. & {[$0.3$, $0.8$, $1.2$]} & {[$0.5$, $0.9$, $1.4$]} & {[$50.5$, $51.2$, $51.7$]}\\
\bottomrule


In [8]:
all_things_router = ['ajax_ping,POST,172.16.0.1:1337', 'ajax_ping,HEAD,172.16.0.1:1337', 'ajax_ping,POST,172.16.0.1/someInvalidPath/', 'ajax_ping,HEAD,172.16.0.1/someInvalidPath/', 'ajax_ping,POST,172.16.0.1', 'ajax_ping,HEAD,172.16.0.1']
flatten(flatten([[[item+'_merged:'+os+'_merged:'+browser for item in all_things_router] for os in ['Mac', 'Linux', 'Win']] for browser in ['Chrome', 'Firefox']]))
# all_things_server = ['ajax_ping,POST,172.16.0.1:1337', 'ajax_ping,HEAD,172.16.0.1:1337', 'ajax_ping,POST,172.16.0.1/someInvalidPath/', 'ajax_ping,HEAD,172.16.0.1/someInvalidPath/', 'ajax_ping,POST,172.16.0.1', 'ajax_ping,HEAD,172.16.0.1']
# flatten(flatten([[[item+'_merged:'+os+'_merged:'+browser for item in all_things_server] for os in ['Mac', 'Linux', 'Win']] for browser in ['Chrome', 'Firefox']]))

['ajax_ping,POST,172.16.0.1:1337_merged:Mac_merged:Chrome',
 'ajax_ping,HEAD,172.16.0.1:1337_merged:Mac_merged:Chrome',
 'ajax_ping,POST,172.16.0.1/someInvalidPath/_merged:Mac_merged:Chrome',
 'ajax_ping,HEAD,172.16.0.1/someInvalidPath/_merged:Mac_merged:Chrome',
 'ajax_ping,POST,172.16.0.1_merged:Mac_merged:Chrome',
 'ajax_ping,HEAD,172.16.0.1_merged:Mac_merged:Chrome',
 'ajax_ping,POST,172.16.0.1:1337_merged:Linux_merged:Chrome',
 'ajax_ping,HEAD,172.16.0.1:1337_merged:Linux_merged:Chrome',
 'ajax_ping,POST,172.16.0.1/someInvalidPath/_merged:Linux_merged:Chrome',
 'ajax_ping,HEAD,172.16.0.1/someInvalidPath/_merged:Linux_merged:Chrome',
 'ajax_ping,POST,172.16.0.1_merged:Linux_merged:Chrome',
 'ajax_ping,HEAD,172.16.0.1_merged:Linux_merged:Chrome',
 'ajax_ping,POST,172.16.0.1:1337_merged:Win_merged:Chrome',
 'ajax_ping,HEAD,172.16.0.1:1337_merged:Win_merged:Chrome',
 'ajax_ping,POST,172.16.0.1/someInvalidPath/_merged:Win_merged:Chrome',
 'ajax_ping,HEAD,172.16.0.1/someInvalidPath/_mer